In [1]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    source BIGINT,
    target BIGINT,
    cost FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT
);
```

### Postgres connection

In [21]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow",
    user="postgres",
    password="postgres")

### Data loading

In [22]:
conn.autocommit = True
cursor = conn.cursor() 

In [26]:
create_table = """
    CREATE TABLE edge_table (
    source BIGINT,
    target BIGINT,
    osmid BIGINT,
    length FLOAT,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
"""

create_extension = """
    CREATE EXTENSION postgis;
    CREATE EXTENSION pgrouting;
"""

### Create table

In [27]:
cursor.execute(create_table)

### Load data from csv

In [28]:
csv_file_name = "/home/pcend/Piotr/kod/lisboa-routing/data/krakow/egde_table_sql.csv"
copy_csv_query = "COPY edge_table FROM STDIN DELIMITER ',' CSV HEADER"

In [29]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r"))

### pgRouting and postgis

In [30]:
cursor.execute(create_extension)

### Search queries

In [31]:
dijkstra = """
    SELECT * FROM pgr_dijkstra(
    'SELECT osmid as id, source, target, length as cost FROM edge_table',
    356926768, 2104495834,
    directed => false);
"""

astar = """
    SELECT * FROM pgr_astar(
    'SELECT osmid as id, source, target, length as cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);
"""

dijkstra_benchmark = """
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT osmid as id, source, target, length as cost FROM edge_table',
    356926768, 2104495834,
    directed => false);
"""

astar_benchmark = """
    EXPLAIN ANALYZE
    SELECT * FROM pgr_astar(
    'SELECT osmid as id, source, target, length as cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);
"""

In [46]:
cursor.execute(dijkstra_benchmark)

In [50]:
result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_dijkstra  (cost=0.25..10.25 rows=1000 width=56) (actual time=991.165..991.179 rows=298 loops=1)',)
('Planning Time: 0.038 ms',)
('Execution Time: 991.206 ms',)


In [51]:
cursor.execute(astar_benchmark)

In [53]:
result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_astar  (cost=0.25..10.25 rows=1000 width=40) (actual time=926.425..926.468 rows=1054 loops=1)',)
('Planning Time: 0.047 ms',)
('Execution Time: 926.517 ms',)


### Disconnect

In [20]:
conn.commit() 
conn.close() 